### Power simulation

Test of comparative statics

In [65]:
import matplotlib.pyplot as plt 
import statsmodels.stats.power as smp 
import statsmodels.api as sm
import pandas as pd
import numpy as np
from scipy.stats import multivariate_normal


ModuleNotFoundError: No module named 'seaborn'

In [75]:
def compute_test_result(n_matching_grp_per_treatment, 
                        n_obs_per_matching_grp,
                        small_coal_worth, 
                        unit, 
                        err_type, 
                        err_scale, 
                        ind_set, 
                        use_treatment_dummies, 
                        cov_type):
    """
    Run a statistical test on data generated by assumed DGP

    Parameters: 
    - experiment set-up
        * n_matching_grp_per_treatment # number of matching groups per treatment/session
        * n_obs_per_matching_grp # number of observations per matching group
        * small_coal_worth # list of treatment values
    - DGP:
        * err_type = "uniform" # choose: "uniform", "norm_corr"
        * err_scale # if err_type == uniform, then the error is in [-err_scale, err_scale], if err_type == norm_corr, then err_scale is a multiplier of the covariance matrix
        * ind_set # list with assumed distribution of "true" values (e.g. [0,0,1,1,2], where 0 = equal split, 1 = shapley, 2 = no coordination)
    - specification:
        * use_treatment_dummies # choose: 0,1
        * unit # choose: "group", "matching_group"
        * cov_type # choose: "cluster", "HC0", "HC1", "HC2", "HC3" (if using group as a unit)
    
    Returns:
    - p value(s) for coefficients on treatment dummies / worth of small coalition
    """
    # Generate independent variables
    n_treat = len(small_coal_worth)
    n_obs = n_obs_per_matching_grp * n_matching_grp_per_treatment * n_treat # total number of group observations
    
    Xval = np.array(sum([n_obs_per_matching_grp * n_matching_grp_per_treatment * [val] for val in small_coal_worth], [])) 
    X = pd.get_dummies(Xval, drop_first= True).values.astype(int) if use_treatment_dummies else Xval 
    X = sm.add_constant(X)

    if unit == "matching_group" or (unit == "group" and cov_type == "cluster"): 
        clusters = np.array(sum([n_obs_per_matching_grp * [cluster] for cluster in range(0, n_matching_grp_per_treatment * n_treat)], [])) 
        X = np.column_stack((X, clusters))

    # Assumed DGP 

    if err_type == "uniform":
        e = np.random.random(n_obs) * 2 * err_scale - err_scale
    elif err_type == "norm_corr": # generate correlated errors within each matching group
        cov = err_scale * np.random.random(size=(n_obs_per_matching_grp,n_obs_per_matching_grp))
        cov = cov + cov.T # make it symmetric 
        cov = np.dot(cov, cov.T) # make it positive semidefinite
        e = multivariate_normal.rvs(mean = np.zeros(n_obs_per_matching_grp), cov = cov, size = n_matching_grp_per_treatment * n_treat).flatten()
        
    es = np.ones(n_obs) * (100//3)  
    sh = (100 + Xval)//3  
    no_coord = np.zeros(n_obs)
    ind = {0: es, 1: sh, 2: no_coord} 

    rand = [np.random.choice(ind_set) for j in range(n_obs)] 
    y = np.array([ind[rand[i]][i] for i in range(n_obs)]) + np.array([0 if rand[i]==2 else e[i] for i in range(n_obs)])
    y = np.maximum(y,0) 
    y = np.minimum(y,100)

    if unit == "matching_group":
        y = np.array([np.mean(y[X[:,-1] == cluster], axis = 0) for cluster in range(0, n_matching_grp_per_treatment * n_treat)])
        X = np.array([np.mean(X[:,0:-1][X[:,-1] == cluster], axis = 0) for cluster in range(0, n_matching_grp_per_treatment * n_treat)])

    # Run a statistical test (only on outcomes with successful coordination)

    y_reg = y[y>0] 
    X_reg = X[y>0]

    if unit == "matching_group":
        results = sm.regression.linear_model.OLS(y_reg, X_reg).fit() 
    elif unit == "group":
        if cov_type == "cluster":
            results = sm.regression.linear_model.OLS(y_reg, X_reg[:, :-1]).fit(cov_type = cov_type, cov_kwds= {"groups": X_reg[:, -1]}) # clustered standard errors
        else:
            results = sm.regression.linear_model.OLS(y_reg, X_reg).fit(cov_type = cov_type) # robust standard errors

    return (results.pvalues[1], results.pvalues[2]) if use_treatment_dummies else results.pvalues[1]


In [66]:
def power_simulation(num_runs, 
                     sig_level, 
                     n_matching_grp_per_treatment, 
                     n_obs_per_matching_grp, 
                     small_coal_worth, 
                     unit, 
                     err_type, 
                     err_scale, 
                     ind_set, 
                     use_treatment_dummies, 
                     cov_type):
    """
        Run a power simulation for our bargaining experiment 
    
        Parameters: 
        - simulation:
            * num_runs 
            * sig_level 
        - experiment set-up
            * n_matching_grp_per_treatment # number of matching groups per treatment/session
            * n_obs_per_matching_grp # number of observations per matching group
            * small_coal_worth # list of treatment values
            * unit # choose: "group", "matching_group"
        - DGP:
            * err_type = "uniform" # choose: "uniform", "norm_corr"
            * err_scale # if err_type == uniform, then the error is in [-err_scale, err_scale], if err_type == norm_corr, then err_scale is a multiplier of the covariance matrix
            * ind_set # list with assumed distribution of "true" values (e.g. [0,0,1,1,2], where 0 = equal split, 1 = shapley, 2 = no coordination)
        - specification:
            * use_treatment_dummies # choose: 0,1
            * cov_type # choose: "cluster", "HC0", "HC1", "HC2", "HC3"
        
        Returns: 
        - Dataframe with p value(s) for each simulation run.
    """
    power_sim_results = pd.DataFrame({'p_value_1': np.zeros(num_runs), 'p_value_2': np.zeros(num_runs)}) if use_treatment_dummies else pd.DataFrame({'p_value': np.zeros(num_runs)}) 

    for run in range(num_runs):
        power_sim_results.loc[run, :] = compute_test_result(n_matching_grp_per_treatment, n_obs_per_matching_grp, small_coal_worth, unit, err_type, err_scale, ind_set, use_treatment_dummies, cov_type)

    print([f"{key}: {value}" for key, value in locals().items() if key not in ["power_sim_results", "run"]])

    if use_treatment_dummies: 
        p_reject_1 = np.mean(power_sim_results['p_value_1'] < sig_level)
        p_reject_2 = np.mean(power_sim_results['p_value_2'] < sig_level)
        print(f"Simulated power for treatment dummy 1: {p_reject_1} for significance level {sig_level}") 
        print(f"Simulated power for treatment dummy 2: {p_reject_2} for significance level {sig_level}") 
    else:
        p_reject = np.mean(power_sim_results['p_value'] < sig_level)
        print(f"Simulated power: {p_reject} for significance level {sig_level}") 

    return power_sim_results

In [77]:
# Fixed parameters

n_matching_grp_per_treatment = 5 # matching groups per treatment/session
n_obs_per_matching_grp = 10 # 2 groups à 5 rounds
small_coal_worth = [10, 30, 90] # different treatments
num_runs = 5000 
sig_level = 0.05

# run power simulations 

# lower bound cases: observations on matching-group level
print("-------------- MATCHING GROUP LEVEL -----------------")
power_simulation(num_runs, sig_level, n_matching_grp_per_treatment, n_obs_per_matching_grp, small_coal_worth, unit = "matching_group", err_type = "norm_corr", err_scale = 10, ind_set = [0,0,0,1,1,1,2], use_treatment_dummies = 1, cov_type = "cluster")
power_simulation(num_runs, sig_level, n_matching_grp_per_treatment, n_obs_per_matching_grp, small_coal_worth, unit = "matching_group", err_type = "norm_corr", err_scale = 10, ind_set = [0,0,0,1,1,1,2], use_treatment_dummies = 0, cov_type = "cluster")
power_simulation(num_runs, sig_level, n_matching_grp_per_treatment, n_obs_per_matching_grp, small_coal_worth, unit = "matching_group", err_type = "uniform", err_scale = 10, ind_set = [0,0,0,1,1,1,2], use_treatment_dummies = 1, cov_type = "cluster")
power_simulation(num_runs, sig_level, n_matching_grp_per_treatment, n_obs_per_matching_grp, small_coal_worth, unit = "matching_group", err_type = "uniform", err_scale = 10, ind_set = [0,0,0,1,1,1,2], use_treatment_dummies = 0, cov_type = "cluster")

power_simulation(num_runs, sig_level, n_matching_grp_per_treatment, n_obs_per_matching_grp, small_coal_worth, unit = "matching_group", err_type = "norm_corr", err_scale = 5, ind_set = [0,1,1,1,1,2], use_treatment_dummies = 1, cov_type = "cluster")
power_simulation(num_runs, sig_level, n_matching_grp_per_treatment, n_obs_per_matching_grp, small_coal_worth, unit = "matching_group", err_type = "norm_corr", err_scale = 5, ind_set = [0,1,1,1,1,2], use_treatment_dummies = 0, cov_type = "cluster")

# upper bound cases: observations on (bargaining-)group level 
print("-------------- BARGAINING GROUP LEVEL -----------------")
power_simulation(num_runs, sig_level, n_matching_grp_per_treatment, n_obs_per_matching_grp, small_coal_worth, unit = "group", err_type = "norm_corr", err_scale = 3, ind_set = [0,0,0,1,1,1,2], use_treatment_dummies = 1, cov_type = "cluster")
power_simulation(num_runs, sig_level, n_matching_grp_per_treatment, n_obs_per_matching_grp, small_coal_worth, unit = "group", err_type = "norm_corr", err_scale = 3, ind_set = [0,0,0,1,1,1,2], use_treatment_dummies = 0, cov_type = "cluster")

power_simulation(num_runs, sig_level, n_matching_grp_per_treatment, n_obs_per_matching_grp, small_coal_worth, unit = "group", err_type = "norm_corr", err_scale = 3, ind_set = [0,1,1,1,1,2], use_treatment_dummies = 1, cov_type = "cluster")
power_simulation(num_runs, sig_level, n_matching_grp_per_treatment, n_obs_per_matching_grp, small_coal_worth, unit = "group", err_type = "norm_corr", err_scale = 3, ind_set = [0,1,1,1,1,2], use_treatment_dummies = 0, cov_type = "cluster")

-------------- MATCHING GROUP LEVEL -----------------
['num_runs: 5000', 'sig_level: 0.05', 'n_matching_grp_per_treatment: 5', 'n_obs_per_matching_grp: 10', 'small_coal_worth: [10, 30, 90]', 'unit: matching_group', 'err_type: norm_corr', 'err_scale: 10', 'ind_set: [0, 0, 0, 1, 1, 1, 2]', 'use_treatment_dummies: 1', 'cov_type: cluster']
Simulated power for treatment dummy 1: 0.0506 for significance level 0.05
Simulated power for treatment dummy 2: 0.0846 for significance level 0.05
['num_runs: 5000', 'sig_level: 0.05', 'n_matching_grp_per_treatment: 5', 'n_obs_per_matching_grp: 10', 'small_coal_worth: [10, 30, 90]', 'unit: matching_group', 'err_type: norm_corr', 'err_scale: 10', 'ind_set: [0, 0, 0, 1, 1, 1, 2]', 'use_treatment_dummies: 0', 'cov_type: cluster']
Simulated power: 0.0972 for significance level 0.05
['num_runs: 5000', 'sig_level: 0.05', 'n_matching_grp_per_treatment: 5', 'n_obs_per_matching_grp: 10', 'small_coal_worth: [10, 30, 90]', 'unit: matching_group', 'err_type: unifor

,p_value
0,4.828504e-03
1,6.188960e-09
2,2.216941e-08
3,4.341509e-05
4,2.777900e-03
...,...
4995,8.152121e-04
4996,2.024948e-05
4997,9.418885e-03
4998,8.719464e-06
